In [7]:
import numpy as np 
import torch
import torch.nn as nn 
import matplotlib.pyplot as plt 
import warnings
warnings.filterwarnings("ignore")
from schrodingerCollect import getData

$iht+0.5hxx+|h|^2h=0$

$x∈[−5,5], t∈[0,π/2]$,

$h(0,x)=2sech(x)$,

$h(t,−5)=h(t,5)$,

$hx(t,−5)=hx(t,5)$

In [8]:
train, h_star, u_star, v_star = getData()

In [11]:
u_star, v_star

(tensor([[0.0270],
         [0.0280],
         [0.0291],
         ...,
         [0.0264],
         [0.0263],
         [0.0263]]),
 tensor([[0.0000],
         [0.0000],
         [0.0000],
         ...,
         [0.0440],
         [0.0438],
         [0.0437]]))

In [13]:
#Make a network for h 
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.fc1 = nn.Linear(2, 16)
        self.fc2 = nn.Linear(16,32)
        self.fc3 = nn.Linear(32,2)
    
    def forward(self,x):
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [17]:
u_star + v_star * 1j

tensor([[0.0270+0.0000j],
        [0.0280+0.0000j],
        [0.0291+0.0000j],
        ...,
        [0.0264+0.0440j],
        [0.0263+0.0438j],
        [0.0263+0.0437j]])

In [ ]:
#Making the Physics Informed NN here, look at `physics` to adjust between PINN and Vanilla NN.
class PINN():
    def __init__(self, X, u, v, h, lb, ub, physics):
        
        self.lb = torch.tensor(lb).float()
        self.ub = torch.tensor(ub).float()
        self.physics = physics
        
        self.x = torch.tensor(X[:, 0:1], requires_grad=True).float()
        self.t = torch.tensor(X[:, 1:2], requires_grad=True).float()
        self.u = torch.tensor(u).float()
        self.v = torch.tensor(v).float()
        self.h = np.vstack([u,v]).T
        
        self.network = Network()
        
        self.optimizer = torch.optim.Adam(self.network.parameters(), lr=0.001)
        
    def makeNetwork(self, x,t):
        X = torch.cat([x,t],1)
        return self.network(X)
    
    def residual(self, x,t):
        h = self.makeNetwork(x,t)
        real = h[:, 0]
        imaginary = h[:, 1] 
        #h outputs a tensor like [u, v] which corresponds to u + iv
        h_t = torch.autograd.grad(h, t, grad_outputs=torch.ones_like(h), retain_graph=True)
        h_x = torch.autograd.grad(h, x, grad_outputs = torch.ones_like(h), retain_graph= True)
        h_xx = torch.autograd.grad(h_x, x, grad_outputs=torch.ones_like(h_x), retain_graph = True)
        
        return h_t * 1j + 0.5 * h_xx + (real**2 + imaginary**2) * h
    
    # def loss(self, x, t):
    #     loss1 = torch.mean((self.makeNetwork(x,t) - self.h)**2)
    #     loss2 = torch.mean(self.residual(x,t)**2)
    #     return loss1 + loss2